In [ ]:
!pip install tonic wandb snntorch
!pip install weave
!wandb login ''

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.7/473.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 6.7 MB/s eta 0:00:00
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import snntorch as snn
import wandb
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- SNN Architecture with Sparse Connectivity ----
class SNN(nn.Module):
    def __init__(self, num_inputs, num_hidden, num_outputs, learn_beta=False, beta=0.95, sparsity=0.8):
        super(SNN, self).__init__()
        self.fc1 = nn.Linear(num_inputs, num_hidden, bias=False)
        self.fc2 = nn.Linear(num_hidden, num_outputs, bias=False)
        self.lif1 = snn.Leaky(beta=beta, learn_beta=learn_beta)
        self.lif2 = snn.Leaky(beta=beta, learn_beta=learn_beta, reset_mechanism='none')

        # Apply sparsity masks to fc1 and fc2
        with torch.no_grad():
            mask1 = torch.rand_like(self.fc1.weight) > sparsity
            mask2 = torch.rand_like(self.fc2.weight) > sparsity
            self.fc1.weight.data *= mask1
            self.fc2.weight.data *= mask2
            self.register_buffer("mask1", mask1)
            self.register_buffer("mask2", mask2)

    def forward(self, x):
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem2_rec = []
        for t in range(x.size(1)):
            cur1 = self.fc1(x[:, t])
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            _, mem2 = self.lif2(cur2, mem2)
            mem2_rec.append(mem2)
        return torch.stack(mem2_rec, dim=1)  # (batch, time, outputs)

# ---- Hybrid Parameter Update (PSO-inspired + adaptive Pool) ----
def hybrid_update(mean, velocity, personal_best, global_best, loss_fn, std, samples, x, y, lr=0.1, acc_threshold=0.95):
    sample_batch = mean + std * torch.randn(samples, *mean.shape).to(mean.device)
    losses = []
    accs = []

    for i in range(samples):
        model = SNN(*x.shape[2:], 10, 2).to(mean.device)
        with torch.no_grad():
            flat_params = sample_batch[i]
            offset = 0
            for p in model.parameters():
                numel = p.numel()
                p.data.copy_(flat_params[offset:offset+numel].view_as(p))
                offset += numel
            output = model(x)
            loss = loss_fn(output.mean(1), y)
            pred = output.mean(1).argmax(1)
            acc = (pred == y).float().mean().item()
            losses.append(loss.item())
            accs.append(acc)

    losses = torch.tensor(losses, device=mean.device)
    accs = torch.tensor(accs, device=mean.device)

    best_idx = torch.argmin(losses)
    if losses[best_idx] < loss_fn(model(x).mean(1), y):
        global_best = sample_batch[best_idx].clone()

    # PSO update
    r1, r2 = torch.rand(2)
    velocity = 0.5 * velocity + 1.5 * r1 * (personal_best - mean) + 1.5 * r2 * (global_best - mean)
    mean = mean + lr * velocity

    # Adaptive Pooling
    if accs[best_idx] < acc_threshold:
        print(f"Adaptive Pooling |", end = ' ')
        topk = sample_batch[torch.argsort(losses)[:samples//4]]
        mean = topk.mean(dim=0)

    # Log batch best performance (optional)
    with torch.no_grad():
        model = SNN(*x.shape[2:], 10, 2).to(mean.device)
        offset = 0
        for p in model.parameters():
            numel = p.numel()
            p.data.copy_(mean[offset:offset+numel].view_as(p))
            offset += numel
        output = model(x)
        pred = output.mean(1).argmax(1)
        acc = (pred == y).float().mean().item()
        print(f"    Batch Accuracy: {acc * 100:.2f}%")
        wandb.log({"train_acc": acc})

    personal_best = mean.clone()
    return mean, velocity, personal_best, global_best

# ---- Annealing Schedulers ----
def get_annealed_param(init, final, current_epoch, total_epochs, mode='exp'):
    if mode == 'linear':
        return final + (init - final) * (1 - current_epoch / total_epochs)
    elif mode == 'exp':
        return final + (init - final) * (0.97 ** current_epoch)
    else:
        return init

# ---- Train Function ----
def train_snn():
    run_name = 'EA_2_classes_randman2'
    config = {
        'nb_input': 100, 'nb_output': 2, 'nb_steps': 50, 'nb_data_samples': 1000,
        'nb_hidden': 10, 'learn_beta': False, 'nb_model_samples': 100,
        'std': 0.05, 'epochs': 20, 'batch_size': 256,
        'loss': 'cross-entropy', 'optimizer': 'Adam', 'lr': 0.01, 'regularization': 'none'
    }

    wandb.init(entity='DarwinNeuron', project='EA-Randman', name=run_name, config=config)

    with torch.no_grad():
        dataset = get_randman_dataset(config['nb_output'], config['nb_input'], config['nb_steps'], config['nb_data_samples'])
        train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, shuffle=False)
        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

        sample_model = SNN(config['nb_input'], config['nb_hidden'], config['nb_output'])
        param_vector = torch.cat([p.flatten() for p in sample_model.parameters()]).detach()
        mean = param_vector.clone()
        velocity = torch.zeros_like(mean)
        personal_best = mean.clone()
        global_best = mean.clone()

        for epoch in range(config['epochs']):
            print(f"Epoch {epoch}")
            current_std = get_annealed_param(init=0.1, final=0.01, current_epoch=epoch, total_epochs=config['epochs'])
            current_samples = int(get_annealed_param(init=1000, final=100, current_epoch=epoch, total_epochs=config['epochs']))
            acc_thresh = get_annealed_param(init=0.90, final=0.98, current_epoch=epoch, total_epochs=config['epochs'])

            for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
                x_batch, y_batch = x_batch.float(), y_batch.long()
                mean, velocity, personal_best, global_best = hybrid_update(
                    mean, velocity, personal_best, global_best,
                    nn.CrossEntropyLoss(), current_std, current_samples,
                    x_batch, y_batch, lr=config['lr'], acc_threshold=acc_thresh
                )

            # ---- Evaluate validation set ----
            val_accs = []
            with torch.no_grad():
                model = SNN(config['nb_input'], config['nb_hidden'], config['nb_output']).to(mean.device)
                offset = 0
                for p in model.parameters():
                    numel = p.numel()
                    p.data.copy_(mean[offset:offset+numel].view_as(p))
                    offset += numel

                for x_val, y_val in val_loader:
                    x_val, y_val = x_val.float().to(mean.device), y_val.long().to(mean.device)
                    output = model(x_val)
                    pred = output.mean(1).argmax(1)
                    acc = (pred == y_val).float().mean().item()
                    val_accs.append(acc)

                val_accuracy = sum(val_accs) / len(val_accs)
                print(f"Validation Accuracy: {val_accuracy:.4f} | std: {current_std:.4f} | samples: {current_samples} | acc_thresh: {acc_thresh:.4f}")
                wandb.log({
                    "val_accuracy": val_accuracy,
                    "epoch": epoch,
                    "std": current_std,
                    "samples": current_samples,
                    "acc_threshold": acc_thresh
                })

            torch.cuda.empty_cache()

train_snn()


Epoch 0
Adaptive Pooling | Epoch 0
    Batch Accuracy: 48.05%
Adaptive Pooling | Epoch 0
    Batch Accuracy: 45.31%
Adaptive Pooling | Epoch 0
    Batch Accuracy: 47.27%
Adaptive Pooling | Epoch 0
    Batch Accuracy: 47.66%
Adaptive Pooling | Epoch 0
    Batch Accuracy: 53.91%
Adaptive Pooling | Epoch 0
    Batch Accuracy: 54.30%
Adaptive Pooling | Epoch 0
    Batch Accuracy: 51.56%
Validation Accuracy: 0.5240 | std: 0.1000 | samples: 1000 | acc_thresh: 0.9000
Epoch 1
Adaptive Pooling | Epoch 0
    Batch Accuracy: 51.17%
Adaptive Pooling | Epoch 0
    Batch Accuracy: 46.88%
Adaptive Pooling | Epoch 0
    Batch Accuracy: 49.61%
Adaptive Pooling | Epoch 0
    Batch Accuracy: 49.22%
    Batch Accuracy: 49.61%
Adaptive Pooling | Epoch 0
    Batch Accuracy: 54.69%
Adaptive Pooling | Epoch 0
    Batch Accuracy: 54.69%
Validation Accuracy: 0.6490 | std: 0.0973 | samples: 973 | acc_thresh: 0.9024
Epoch 2
Adaptive Pooling | Epoch 0
    Batch Accuracy: 74.61%
Adaptive Pooling | Epoch 0
    Batch

KeyboardInterrupt: 